The task in this notebook is mortality prediction on 48hrs using clinical notes.

The positive class is mortality and the negative survive.

We define preprocessing, and test, training classes in python and pytorch. You can use the preivous practical as an exmaple to complete this notebook

# Exercises

1. Check the file `test_text_data_2/train/20/20_notes.txt`
   - Plot the bar char of the most top-35 most common words
   - Which words do seem most relevant to you?
1. For the mortality prediction model based on average word embeddings (Bag-of-Words, BoW)  
  - Implement the missing layers in the constructor
  - Implement the forward pass, note: the batch for loop is different, and you will need to call the function create sentence batch
      - use mean of word embeddings to aggregate features for the classifier
      - **EXTRA** use average of the sentence (sum over the word dimension / sequence lenght) instead of mean to agregate features.  note: that the sequence lenght (seq_len) variable contains the actual size of each sentence in the batch. In addition, to define the correct size of the seq_len matrix you have to expand it with: `seq_len.unsqueeze(-1)`
      - **EXTRA** compare the AUC-ROC on test betweent the mean and average arregation
  - Implement model selection 
  - Print the test results for all the metrics
  - Plot AUC and calibration curves (for test data)
  - **EXTRA** run 3 random BoW with mean models (set seed to None) and report mean and std of the AUC-ROC and AUC-PR. what do you observe?

2. Unfreeze the word embeddings for the BoW model with mean 
  - Compare the AUC-ROC on the test with the freeze model. what do you observe? Which one is better?
  - Change the learning rate to 1e-4 for the unfreeze model and compare AUC-ROC with the previous models. what do you observe? Which one is better?

3. Implement the LSTM model instead of BoW
  - Add a LSTM layer into the constructor. NOTE: we recommend w_max is 3000 words, but you can try to push the GPU (also it will be slower). A longer sequence would make RNNs unfeasible.
  - Use the best setting from the previous experiment for freeze or unfreeze word embeddings
  - Apply the LSTM after the word embedding.
  - Use mean as the aggeration of the LSTM ouputs.
  - Print the test results for all the metrics
  - Plot AUC and calibration curves (for test data)
  - Compare the AUC-ROC on the test with the best BoW model. what do you observe? Which one is better?
  - Use bidirectional LSTM
    - Compare the AUC-ROC on the test with the LSTM. what do you observe? Which one is better?
4. Compare the AUC-ROC across, your best structued, BoW and LSTM models. what do you observe? Which one is better?
5. **EXTRA** Train your own word embeddings with the extra glove example notebook.



## Download data

In [ ]:
#dowload csv files from gdrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#download file from gdrive with the id for share file
# create share link for tar.gz file  and copy id
#https://drive.google.com/file/d/1E279yz7ZiZmok6qOYWlrWku1w7F13T5Q/view?usp=sharing
file_id = '1E279yz7ZiZmok6qOYWlrWku1w7F13T5Q' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('test_text_data_2.tar.gz')

In [ ]:
#extract data
!tar -xzf test_text_data_2.tar.gz

In [ ]:
#download file with word embeddings
# https://drive.google.com/file/d/1NV3TIOF0P4cKfZ0Iwl8F3bwMk5-FfUiQ/view?usp=sharing
file_id = '1NV3TIOF0P4cKfZ0Iwl8F3bwMk5-FfUiQ' # URL id. 
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('mimic_vectors_training.100d.txt')

In [ ]:
#check size of vacabulary
!wc -l mimic_vectors_training.100d.txt

In [ ]:
#download discretizer config
#TODO add config into dataset
from google.colab import files
uploaded = files.upload()

Saving discretizer_config.json to discretizer_config.json


In [ ]:
#download normalizer config
#TODO add config into dataset
from google.colab import files
uploaded = files.upload()

Saving norm_start_time_zero.normalizer to norm_start_time_zero.normalizer


## upload utils 
**Upload mimic__utils_text.py**
For reading csv, normalize data, and imputation.
imputation techinique is previous, there are other imputation methods avilable.

In [ ]:
#download normalizer config
#TODO add config into dataset
from google.colab import files
uploaded = files.upload()

Saving mimic_utils_text.py to mimic_utils_text.py


In [ ]:
# install stop words, you can use the option in the create batch funciton to filter  funcitonal words
!pip install stop_words

## Import libraries

In [ ]:
#import python and pytorch libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import codecs
import os
import sys
import numpy as np
import logging
import tempfile
import shutil
import pickle
import platform
import json
from datetime import datetime
from nltk.corpus import stopwords
from stop_words import get_stop_words
from collections import defaultdict
import string
import random
from __future__ import absolute_import
from __future__ import print_function
from sklearn import metrics
from mimic_utils_text import InHospitalMortalityReader, Discretizer, Normalizer, read_chunk

# Figures ROC and calibration curve
import matplotlib.pyplot as plt
import sys
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms
import pickle
from sklearn import metrics

## Pytorch Dataset

We define a vocabulary, dataset, a collate function and create batch 


In [ ]:
# vocabulary class to upload word2vec into pytorch
# default tokens
UNK_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"
SOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"


class Vocabulary:
    """
        Creates a vocabulary from a word2vec file. 
    """
    def __init__(self):
      # we add special tokens for padding, unknown word, start of sentence and end of sentence
      # idx_to_word is a dictionary of ids to words
      # word_to_idx is a dictionary from words to ids
        self.idx_to_word = {0: PAD_TOKEN, 1: UNK_TOKEN, 2: SOS_TOKEN, 3: EOS_TOKEN}
        self.word_to_idx = {PAD_TOKEN: 0, UNK_TOKEN: 1, SOS_TOKEN: 2, EOS_TOKEN: 3}
        self.word_freqs = {}
       
    
    def __getitem__(self, key):
        return self.word_to_idx[key] if key in self.word_to_idx else self.word_to_idx[UNK_TOKEN]
    
    def word(self, idx):
        return self.idx_to_word[idx]
    
    def size(self):
        return len(self.word_to_idx)
    
    
    def from_data(input_file, vocab_size, emb_size):
      # we read the word embeddings file into memory 
      #and create the unknown word with the average of all the vocabulary
        vocab = Vocabulary()
        vocab_size = vocab_size + len(vocab.idx_to_word)
        weight = np.zeros((vocab_size, emb_size))
        with codecs.open(input_file, 'rb')  as f:
         
          for l in f:
            line = l.decode().split()
            token = line[0] # first position token
            if token not in vocab.word_to_idx:
              idx = len(vocab.word_to_idx)
              vocab.word_to_idx[token] = idx
              vocab.idx_to_word[idx] = token
            
              vect = np.array(line[1:]).astype(np.float) #all cols are the embeddings
              weight[idx] = vect
          # average embedding for unk word
          avg_embedding = np.mean(weight, axis=0)
          weight[1] = avg_embedding
                            
        return vocab, weight

In [ ]:

# pytroch class for reading data into batches
class MIMICTextDataset(Dataset):
    """
       Loads a list of sentences into memory from a text file,
       split by newlines. 
    """
    def __init__(self, reader, discretizer, normalizer, 
            notes_output='sentence', max_w=25, max_s=500, max_d=500,
            target_repl=False, batch_labels=False):
        self.data = []
        self.y  = []
        self.max_w = max_w
        self.max_s = max_s
        self.max_d = max_d
        N = reader.get_number_of_examples()
        #if small_part:
        #    N = 1000
        ret = read_chunk(reader, N)
        data = ret["X"]
        notes_text = ret["text"] # we also load the text data from MIMIC
        notes_info = ret["text_info"] # extra info form the text like category of a note
        ts = ret["t"]
        labels = ret["y"]
        names = ret["name"]
        data = [discretizer.transform(X, end=t)[0] for (X, t) in zip(data, ts)]
        if normalizer is not None:
            data = [normalizer.transform(X) for X in data]
        
        #notes into list of sentences, docs, etc..
        self.notes = []
        tmp_data = []
        tmp_labels = []
        if notes_output == 'sentence':
            # [N, W] patients, words
            # we exclude patients that have more tan max_w words
            for patient_notes, _x, l  in zip(notes_text, data, labels):
                tmp_notes = []
                for doc in sorted(patient_notes):
                    sentences = patient_notes[doc]
                    for sentence in sentences:
                        #print(sentence)
                        tmp_notes.extend(sentence)
                if len(tmp_notes) > 0 and len(tmp_notes) <= self.max_w:
                    #print(tmp_notes)
                    self.notes.append(' '.join(tmp_notes))
                    #self.notes.append(tmp_notes)
                    tmp_data.append(_x)
                    tmp_labels.append(l)
                #elif len(tmp_notes) > 0:
                #    self.notes.append(' '.join(tmp_notes[:self.max_w]))
                #    tmp_data.append(_x)
        elif notes_output == 'sentence-max':
             # [N, W] patients, words
             # we cut notes of each patient up to max_w words
            for patient_notes, _x, l  in zip(notes_text, data, labels):
                tmp_notes = []
                for doc in sorted(patient_notes):
                    sentences = patient_notes[doc]
                    for sentence in sentences:
                        #print(sentence)
                        tmp_notes.extend(sentence)
                if len(tmp_notes) > 0 and len(tmp_notes) <= self.max_w:
                    #print(tmp_notes)
                    self.notes.append(' '.join(tmp_notes))
                    #self.notes.append(tmp_notes)
                    tmp_data.append(_x)
                    tmp_labels.append(l)
                elif len(tmp_notes) > 0:
                    self.notes.append(' '.join(tmp_notes[:self.max_w]))
                    tmp_data.append(_x)
                    tmp_labels.append(l)

        elif notes_output == 'doc':
            # [N, S, W] patients, sentences, words
            # we cut notes into max sentences and each sentence into max words
            for patient_notes,  _x, l in zip(notes_text, data, labels):
                tmp_notes = []
                for doc in sorted(patient_notes):
                    sentences = patient_notes[doc]
                    for sentence in sentences:
                        if len(sentence) > 0 and len(sentence) <= max_w:
                            tmp_notes.append(sentence)
                        elif len(sentence) > 0:
                            tmp_notes.append(sentence[:max_w])
                if len(tmp_notes) > 0 and len(tmp_notes) <= max_s:
                    self.notes.append(tmp_notes)
                    tmp_data.append(_x)
                    tmp_labels.append(l)
                elif len(tmp_notes) > 0:
                    self.notes.append(tmp_notes[:max_s])
                    tmp_data.append(_x)
                    tmp_labels.append(l)
        
#
        self.x = np.array(tmp_data, dtype=np.float32)   
        self.T = self.x.shape[1]
        if batch_labels:
            self.y = np.array([[l] for l in tmp_labels], dtype=np.float32)
        else:
            self.y = np.array(tmp_labels, dtype=np.float32)


    def _extend_labels(self, labels):
        # (B,)
        labels = labels.repeat(self.T, axis=1)  # (B, T)
        return labels

    def __len__(self):
        # overide len to get number of instances
        return len(self.x)

    def __getitem__(self, idx):
        # get words and label for a given instance index
        # note now we have 2 sources or modalities of data
        # structured variables, text and labels
        return self.x[idx], self.notes[idx], self.y[idx]


In [ ]:
#collate and create batch for sentences 

# the following function takes a batch of words [B, W] and creates a batch to the GPU
# the batch is transfomred into ids and padded 
def create_sentence_batch(sentences, vocab, device, stopwords=False):
    """
    Converts a list of sentences to a padded batch of word ids. Returns
    an input batch, output tags, a sequence mask over
    the input batch, and a tensor containing the sequence length of each
    batch element.
    :param sentences: a list of sentences, each a list of token ids
    :param vocab: a Vocabulary object for this dataset
    :param device: 
    :returns: a batch of padded inputs,  mask, lengths
    """
    # we can skip stop words from a sequence 
    if stopwords:
        tok = np.array([_remove_stopwords(sen) for sen in sentences])
    else:
        tok = np.array([sen.split() for sen in sentences])
    #tok = np.array([sen[0] for sen in sentences])
    seq_lengths = [len(sen) for sen in tok]
    max_len = max(seq_lengths)
    pad_id = vocab[PAD_TOKEN]
    unk_id = vocab[UNK_TOKEN]

    pad_id_input = []
    #pad and find ids for words given the word2vec vocab
    #print(tok)
    for idx, sen in enumerate(tok):
      tmp_sent = []
      for t in range(max_len):
        if t < seq_lengths[idx]:
          try:
            token_id = vocab[sen[t]]
          except KeyError:
            token_id = unk_id
        else:
          token_id = pad_id
        tmp_sent.append(token_id)
      pad_id_input.append(tmp_sent) 

    
    # Convert everything to PyTorch tensors.
    batch_input = torch.tensor(pad_id_input)
    seq_mask = (batch_input != vocab[PAD_TOKEN])
    seq_length = torch.tensor(seq_lengths)
    
    # Move all tensors to the given device.
    batch_input = batch_input.to(device)
    seq_mask = seq_mask.to(device)
    seq_length = seq_length.to(device)
    
    return batch_input, seq_mask, seq_length





## Evaluation metrics

In [ ]:
# eval metrics
def print_metrics_binary(y_true, predictions, logging, verbose=1):
    predictions = np.array(predictions)
    if len(predictions.shape) == 1:
        predictions = np.stack([1 - predictions, predictions]).transpose((1, 0))
    cf = metrics.confusion_matrix(y_true, predictions.argmax(axis=1))
    if verbose:
        logging.info("confusion matrix:")
        logging.info(cf)
    cf = cf.astype(np.float32)

    acc = (cf[0][0] + cf[1][1]) / np.sum(cf)
    prec0 = cf[0][0] / (cf[0][0] + cf[1][0])
    prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
    rec0 = cf[0][0] / (cf[0][0] + cf[0][1])
    rec1 = cf[1][1] / (cf[1][1] + cf[1][0])
    auroc = metrics.roc_auc_score(y_true, predictions[:, 1])

    (precisions, recalls, thresholds) = metrics.precision_recall_curve(y_true, predictions[:, 1])
    auprc = metrics.auc(recalls, precisions)
    minpse = np.max([min(x, y) for (x, y) in zip(precisions, recalls)])

    if verbose:
        logging.info("accuracy = {0:.3f}".format(acc))
        logging.info("precision class 0 = {0:.3f}".format(prec0))
        logging.info("precision class 1 = {0:.3f}".format(prec1))
        logging.info("recall class 0 = {0:.3f}".format(rec0))
        logging.info("recall class 1 = {0:.3f}".format(rec1))
        logging.info("AUC of ROC = {0:.3f}".format(auroc))
        logging.info("AUC of PRC = {0:.3f}".format(auprc))
       

    return {"acc": acc,
            "prec0": prec0,
            "prec1": prec1,
            "rec0": rec0,
            "rec1": rec1,
            "auroc": auroc,
            "auprc": auprc}

## Understand the data

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# read the file
with open('test_text_data_2/train/20/20_note.txt','r') as f:
  txt = f.read()
# Exclude headers
txt = re.sub("[0-9]+,[0-9]+,[0-9][0-9][0-9][0-9]-[0-1][0-9]-[0-3][0-9]\s[0-2][0-9]:[0-5][0-9]:[0-9][0-9],[0-9][0-9][0-9][0-9]-[0-1][0-9]-[0-9][0-9]\s[0-9][0-9]:[0-9][0-9]:[0-9][0-9],[0-9]+\.[0-9][0-9]+,[0-9]+,[0-9]+\.[0-9]+\n", "", txt)
# tokenize
txt = txt.split('\n')
# exclude puctuation
txt = filter(lambda x: not re.match(r'^,|:|to|%|\.|\*+|\[|\]|\(|\)|\^|\s*$', x), txt)
# exclude stopwords
stop_words = set(stopwords.words('english'))
filtered_sentence = [w for w in txt if not w.lower() in stop_words]
# Your code here
# Hint: you can use nltk.FreqDist and nltk.FreqDist.plot()

## Training

Here we define the model, loss and training loop

In [ ]:
#model Bow with a single sequence for a patient

class BoWText(nn.Module):

    def __init__(self, vocab_size, label_size, emb_size, hidden_size, dropout=0.2, model_w2vec=None, bidirectional=False):

        super().__init__()
        self.bidirectional = bidirectional
        # This is how the network uploads embeddings into the lookup layer
        # numpy vectors into tensor
        weights = torch.FloatTensor(model_w2vec)
        # this embedder will be the encoder for the text notes 
        # freeze true means that is not going to update the parameters of the word embeddings
        self.embedder = nn.Embedding.from_pretrained(weights, freeze=True) 
        # your code here
        # add hidden combination layer
        # add projection layer
        # add dropout layer
        # add relu activation function


    def forward(self, x, seq_mask, seq_len): #x=notes
        # your code here
        # Compute word embeddings
        # [B,W,E] B=patient, M=sentence, E=embedings
        # apply dropout 
        # apply a hidden layer
        # [B, W, hid_size]
        # apply mean to aggregate features
        # project features into logits 
        # [B,1] 
        
        return logits

In [ ]:
#eval model
def eval_model(model, dataset, device, vocab):
    model.eval()
    sigmoid = nn.Sigmoid()
    with torch.no_grad():
        y_true = []
        predictions = []
        for _, notes, labels  in dataset:
            labels = labels.to(device)
            x_notes, seq_mask, seq_len = create_sentence_batch(notes, 
                    vocab, 
                    device, 
                    stopwords=False)

            logits =  model(x_notes, seq_mask, seq_len)
            probs = sigmoid(logits)
            #_, predicted = torch.max(probs.data, 1)
            #y_hat_class = np.where(probs.data<0.5, 0, 1)
            predictions += [p.item() for p in probs]#y_hat_class.squeeze()
            y_true += [y.item() for y in labels]
    #print(predictions)
    #print(y_true)
    results = print_metrics_binary(y_true, predictions, logging)
    return results, predictions, y_true

In [ ]:
def train(args):
  mode = 'train'
  hidden_size = args['dim']
  dropout = args['dropout']
  batch_size = args['batch_size']
  learning_rate = args['lr']
  num_epochs = args['epochs']
  emb_size = args['emb_size']
  aggregation_type = args['aggregation_type']
  bidirectional_encoder = args['bidirectional'] # TODO add into args
  seed = args['seed']
  steps = args['steps']
  data = args['data']
  notes = args['notes']
  word2vec = args['word2vec']
  max_w = args['max_w']
  timestep = args['timestep']
  normalizer_state = args['normalizer_state']
  if seed:
      torch.manual_seed(seed)
      np.random.seed(seed)
  device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")   
  
  logging.basicConfig(level=logging.INFO, 
          format='%(asctime)s %(message)s', 
          datefmt='%Y-%m-%d %H:%M:%S',
          )
  
  vocab, weight = Vocabulary.from_data(word2vec, 130212, emb_size) #glove size vocabulary, emb size
    #print(vocab)
    #rint(vocab["<unk>"])

  train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'train'),
                                        notes_dir=notes,  
                                        listfile=os.path.join(data, 'train_listfile.csv'),
                                         period_length=48.0)

  val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'train'),
                                       notes_dir=notes,
                                       listfile=os.path.join(data, 'val_listfile.csv'),
                                       period_length=48.0)

  discretizer = Discretizer(timestep=float(timestep),
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')
  discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
  cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

  normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
  normalizer_state = normalizer_state
  if normalizer_state is None:
      normalizer_state = 'norm_start_time_zero.normalizer'
     
  normalizer.load_params(normalizer_state)

  # sentence-max option proces notes into single sequence
  train_dataset = MIMICTextDataset(train_reader, 
                discretizer, 
                normalizer, 
                batch_labels=True,
                max_w=max_w,
                notes_output='sentence-max')
  train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  
  val_dataset = MIMICTextDataset(val_reader, 
            discretizer, 
            normalizer, 
            batch_labels=True,
            max_w=max_w,
            notes_output='sentence-max')
  val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
  
  # Define the classification model.
  model = BoWText(vocab_size=vocab.size(),
                        label_size=1, 
                        emb_size=emb_size, 
                        hidden_size=hidden_size,
                        dropout=dropout,
                        model_w2vec=weight)

  model = model.to(device)
  logging.info(args)
  logging.info(model)

  # Define optimizer
  optimizer = Adam(model.parameters(), lr=learning_rate) 
   
  criterion = nn.BCEWithLogitsLoss()

  # path to best model save on disk
  best_model = 'best_model.pt'
  best_val_auc = 0.

  results = []

  step = 0
  num_batches = 0
  # your code here
  #training loop
  # loop over the epochs
  #for epoch_num... 
      #loss_batch = .0
      #num_batches = 0
      # loop over mini-batches note now we ignore x and use notes as input to create batch
      #for _, notes, labels  in train_dl:
          # use create sentence batch to make batch tensors for input for your model
          #x_notes, seq_mask, seq_len = create_sentence_batch(notes, 
          #          vocab, 
          #          device, 
          #          stopwords=False)
          
          # run forward
          # Backpropagate and update the model weights.
         
        
          # Every x steps we evaluate the model and report progress.
          if step % steps == 0:
              logging.info("epoch (%d) step %d: training loss = %.2f"% 
                 (epoch_num, step, loss_batch/num_batches))
            
            
          step += 1
        
        
      metrics_results, _, _ = eval_model(model,
                                    val_dl,
                                    device,
                                    vocab)
      metrics_results['epoch'] = epoch_num
      results.append(metrics_results)
      #Your code here
      # model selection
      


In [ ]:
#execute training 
#define hyperparameters

args = {'dim':128,
        'dropout':0.2,
        'batch_size':64,
        'lr':1e-3,
        'epochs':20,
        'emb_size':100, # from our pretrained embeddings
        'aggregation_type':'mean',
        'bidirectional':False,
        'seed':42, # set to None for random model
        'steps':50,
        'data':'test_text_data_2/in-hospital-mortality',
        'notes': 'test_text_data_2/train',
        'word2vec': '', #file with word embeddings
        'max_w': 10000, # note change for LSTM and BiLSTM models
        'timestep':1.0,
        'imputation':'previous',
        'normalizer_state':None}
train(args)

## Test

Here we use the best validation model and run in test.

In [ ]:
#test

def test(args):
    # define trainning and validation datasets
    mode = 'test'
    hidden_size = args['dim']
    dropout = args['dropout']
    batch_size = args['batch_size']
    emb_size = args['emb_size']
    best_model = args['best_model']
    data = args['data']
    notes = args['notes']
    word2vec = args['word2vec']
    max_w = args['max_w']
    timestep = args['timestep']
    aggregation_type = args['aggregation_type']
    bidirectional_encoder = args['bidirectional'] # TODO add into args
    device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")   
    # 1. Get a unique working directory 
    
    logging.basicConfig(level=logging.INFO, 
            format='%(asctime)s %(message)s', 
            datefmt='%Y-%m-%d %H:%M:%S')
    
    vocab, weight = Vocabulary.from_data(word2vec, 130212, emb_size) #glove 

    test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data, 'test'),
                                         listfile=os.path.join(data, 'test_listfile.csv'),
                                         notes_dir=notes, 
                                         period_length=48.0)

    
    discretizer = Discretizer(timestep=float(timestep),
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

    discretizer_header = discretizer.transform(test_reader.read_example(0)["X"])[1].split(',')
    cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

    normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
    normalizer_state = args['normalizer_state']
    if normalizer_state is None:
        #normalizer_state = 'ihm_ts{}.input_str:{}.start_time:zero.normalizer'.format(args.timestep, args.imputation)
        #normalizer_state = os.path.join(os.path.dirname(__file__), normalizer_state)
        normalizer_state = 'norm_start_time_zero.normalizer'
    normalizer.load_params(normalizer_state)

    # Read data
    vocab, weight = Vocabulary.from_data(word2vec, 107647, emb_size) #glove 107647 , 300)
    # sentence option proces notes into single sequences
    test_dataset = MIMICTextDataset(test_reader, 
            discretizer, 
            normalizer, 
            batch_labels=True,
            max_w=max_w,
            notes_output='sentence-max')
 
    test_dl =  DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


    # Define the classification model.
    model = BoWText(vocab_size=vocab.size(),
                        label_size=1, #label size = 1 because of the binary nature of the predictions(classification)
                        emb_size=emb_size, 
                        hidden_size=hidden_size,
                        dropout=dropout,
                        model_w2vec=weight)

    model.load_state_dict(torch.load(best_model))
    logging.info(model)
    model = model.to(device)

    metrics_results, pred_probs, y_true = eval_model(model,
                                test_dl,
                                device,
                                vocab)
    return metrics_results, pred_probs, y_true
            

In [ ]:
# Run test on best validation model
# note use same hyper parameters as in training
args = {'best_model':'best_model.pt',
        'dim':128,
        'dropout':0.2,
        'batch_size':16,
        'word2vec':'', #file with word embeddings
        'emb_size':100,
        'aggregation_type':'mean',
        'bidirectional':False,
        'data':'test_text_data_2/in-hospital-mortality',
        'notes':'test_text_data_2/test',
        'timestep':1.0,
        'max_w':10000,
        'imputation':'previous',
        'normalizer_state':None}
metrics_results, pred_probs, y_true = test(args)

## Plots

ROC and calibrations curves

In [ ]:
# roc curve
bow_fpr, bow_tpr, bow_thresholds = metrics.roc_curve(y_true, pred_probs)

# plot the roc curve for the model
plt.figure()
plt.ylim(0., 1.0)
plt.xlim(0.,1.0)
plt.plot(bow_fpr, bow_tpr, marker='.', label='BoW', color='darkorange')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
# show the legend
plt.legend()
plt.show()

In [ ]:
#calibration curve
bow_y, bow_x = calibration_curve(y_true, pred_probs, n_bins=10)
plt.figure()
plt.ylim(0., 1.0)
plt.xlim(0.,1.0)
#fig, ax = plt.subplots()
# only these two lines are calibration curves
plt.plot(bow_x,bow_y, marker='^', linestyle="", markersize=7, label='BoW', color='darkorange')

plt.plot([0, 1], [0, 1], color='navy', linestyle='--')

plt.xlabel('Mean predicted value')
plt.ylabel('Fraction of positives')
plt.legend()
plt.show()

#plt.show()
